# Part 2: Part 2: Simple Text Processing - Tokenization, Lemmatization, Word Frequency, Vectorization (20 pts)

## Processing Speeches with SpaCy

In [1]:
# load in data and import basic packages

import spacy
from tqdm import tqdm
from collections import Counter

nlp = spacy.load("en_core_web_sm")